In [2]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV,KFold,train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.externals import joblib 
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA
from sklearn.cross_validation import cross_val_score

from sklearn.linear_model import Ridge,Lasso,RidgeCV,LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

import xgboost as xgb
from lightgbm import LGBMRegressor
import math
%matplotlib inline

pd.set_option('display.max_colwidth',1000)
pd.set_option('display.height',1000)
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)
pd.set_option('display.width',1000)

height has been deprecated.



In [3]:
# 数据预处理
def drop_all_outlier(df):
    df.drop_duplicates(df.columns.drop('ID'), keep='first', inplace=True)
    df.drop(df[(df.电压A > 800) | (df.电压A < 500)].index,inplace=True)
    df.drop(df[(df.电压B > 800) | (df.电压B < 500)].index,inplace=True)
    df.drop(df[(df.电压C > 800) | (df.电压C < 500)].index,inplace=True)
    df.drop(df[(df.现场温度 > 30) | (df.现场温度 < -30)].index,inplace=True)
    df.drop(df[(df.转换效率A > 100)].index,inplace=True)
    df.drop(df[(df.转换效率B > 100)].index,inplace=True)
    df.drop(df[(df.转换效率C > 100)].index,inplace=True)
    df.drop(df[(df.功率A > 6000)].index,inplace=True)
    df.drop(df[(df.功率B > 6000)].index,inplace=True)
    df.drop(df[(df.功率C > 6000)].index,inplace=True)
    df.drop(df[(df.平均功率 > 6000)].index,inplace=True)
    df.drop(df[(df.风向 > 360)].index,inplace=True)
    df.drop(df[(df.风速 > 20)].index,inplace=True)
    return df
# 生成数据
def generate_train_data(train_data, test_data, poly=False, select=False):
    y = train_data['发电量']
    X = train_data.drop(['发电量','ID'], axis=1)
    sub_data = test_data.drop(['ID'], axis=1)
    
    polynm = None
    if poly:
        from sklearn.preprocessing import PolynomialFeatures
        polynm = PolynomialFeatures(degree=2, interaction_only=False)
        X = polynm.fit_transform(X)
        sub_data = polynm.transform(sub_data)
        
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
    
    sm = None
    if select:
        from sklearn.feature_selection import SelectFromModel
        sm = SelectFromModel(GradientBoostingRegressor(random_state=2))
        X_train = sm.fit_transform(X_train, y_train)
        X_test = sm.transform(X_test)
        sub_data = sm.transform(sub_data)
        
    return X_train, X_test, y_train, y_test, sub_data, sm, polynm

def cal_score(mse):
    if isinstance(mse, float):
        return 1 / (1 + math.sqrt(mse))
    else:
        return np.divide(1, 1 + np.sqrt(mse))
#  定义交叉验证函数  
def cross_validation_test(models, train_X_data, train_y_data, cv=5):
    model_name, mse_avg, score_avg = [], [], []
    for i, model in enumerate(models):
        print(i + 1,'- Model:', str(model).split('(')[0])
        model_name.append(str(i + 1) + '.' + str(model).split('(')[0])
        nmse = cross_val_score(model, train_X_data[i], train_y_data[i], cv=cv, scoring='neg_mean_squared_error')
        
        avg_mse = np.average(-nmse)
        scores = cal_score(-nmse)
        avg_score = np.average(scores)
        mse_avg.append(avg_mse)
        score_avg.append(avg_score)
        print('MSE:', -nmse)
        print('Score:', scores)
        print('Average XGB - MSE:', avg_mse, ' - Score:', avg_score, '\n')
    res = pd.DataFrame()
    res['Model'] = model_name
    res['Avg MSE'] = mse_avg
    res['Avg Score'] = score_avg
    return res

# def add_newid(df):
#     ID = df["ID"]
#     df["new_id"]=(np.mod(ID,205))
#     return df
# def add_avg(df):
#     array = np.array(df["平均功率"])
#     newarray=[]
#     num = 0
#     for i in np.arange(len(array)):
#         for j in np.arange(10):
#             if i<10:
#                 num = (array[j-1]+array[j-2]+array[j-3])/3
#             if i>=10:
#                 num = (array[i-1]+array[i-2]+array[i-3]+array[i-5]+array[i-6]+array[i-7]+array[i-8]+array[i-9])/9
#         newarray.append(num)
#     df["old平均功率"] = newarray
#     return df
def add_avg(df):
    array = np.array(df["平均功率"])
    newarray=[]
    num = 0
    for i in np.arange(len(array)):
        if i<3:
            num = array[i]
        if i>=3:
            num = (array[i-1]+array[i-2]+array[i-3])/3
        if i>=9:
            num = (array[i-1]+array[i-2]+array[i-3]+array[i-5]+array[i-6]+array[i-7]+array[i-8]+array[i-9])/10
        newarray.append(num)
    df["old平均功率"] = newarray
    return df

In [4]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
# feature = ['ID','电压A','电压B','电压C','电流A','电流B','电流C','功率A','功率B','功率C','平均功率']
# feature_train = feature+['发电量']
# train_data = train_data[feature_train]
# test_data = test_data[feature]
# train_data = add_newid(train_data)
# test_data = add_newid(test_data)
# train_data = add_avg(train_data)
# test_data = add_avg(test_data)
df_result = pd.DataFrame()
df_result['ID'] = list(test_data['ID'])
special_missing_ID = test_data[test_data[(test_data == 0) | (test_data == 0.)].count(axis=1) > 13]['ID']

In [5]:
cleaned_train_data = train_data.copy()
cleaned_train_data = drop_all_outlier(cleaned_train_data)

cleaned_sub_data = test_data.copy()
cleaned_sub_data = drop_all_outlier(cleaned_sub_data)
cleaned_sub_data_ID = cleaned_sub_data['ID']

# all_data = pd.concat([train_data, test_data], axis=0).sort_values(by='ID').reset_index().drop(['index'], axis=1)
# bad_feature = ['ID', '功率A', '功率B', '功率C', '平均功率', '现场温度', '电压A', '电压B', '电压C', '电流B', '电流C', '转换效率', '转换效率A', '转换效率B', '转换效率C']
# bad_index = all_data[bad_feature][
#     (all_data[bad_feature] > all_data[bad_feature].mean() + 2 * all_data[bad_feature].std()) | 
#     (all_data[bad_feature] < all_data[bad_feature].mean() - 2 * all_data[bad_feature].std())
# ].dropna(how='all').index

all_data  = pd.concat([train_data, test_data], axis=0).sort_values(by='ID').reset_index().drop(['index'], axis=1)
# bad_feature = ['ID','功率A', '功率B', '功率C', '平均功率', '现场温度', '电压A', '电压B', '电压C', '电流B', '电流C', '转换效率', '转换效率A', '转换效率B', '转换效率C']
bad_feature = ['ID','电压A','电压B','电压C','电流A','电流B','电流C','功率A','功率B','功率C','平均功率']

bad_index1 = all_data[bad_feature][
    (all_data[bad_feature] > all_data[bad_feature].mean() + 2 * all_data[bad_feature].std()) | 
    (all_data[bad_feature] < all_data[bad_feature].mean() - 2 * all_data[bad_feature].std())
].dropna(how='all').index

bad_index2 = all_data[
    ((all_data['电压A']<500)&(all_data['电压A']!=0))|
    ((all_data['电压B']<500)&(all_data['电压B']!=0))|
    ((all_data['电压C']<500)&(all_data['电压C']!=0))].index
bad_index = pd.Int64Index(list(bad_index1)+list(bad_index2))
# all_data.loc[np.concatenate([bad_index -1,bad_index,bad_index+1])].sort_values(by='ID', ascending=True)

nn_bad_data = all_data.loc[np.concatenate([bad_index - 1, bad_index, bad_index + 1])].sort_values(by='ID', ascending=True).drop_duplicates()
bad_data = all_data.loc[bad_index].sort_values(by='ID', ascending=True)

In [6]:
# 上下记录均值替代异常值
for idx, line in bad_data.iterrows():
    ID = line['ID']
    col_index = line[bad_feature][ 
        (line[bad_feature] > all_data[bad_feature].mean() + 3 * all_data[bad_feature].std())| 
        (line[bad_feature] < all_data[bad_feature].mean() - 3 * all_data[bad_feature].std())
    ].index
    index = all_data[all_data['ID'] == ID].index
    
    before_offset = 1
    while (idx + before_offset)in bad_index:
        before_offset += 1

    after_offset = 1
    while (idx + after_offset) in bad_index:
        after_offset += 1
    
    replace_value = (all_data.loc[index - before_offset, col_index].values + all_data.loc[index + after_offset, col_index].values) / 2
    all_data.loc[index, col_index] = replace_value[0]

In [7]:
#拆分数据
train_data = all_data.drop(all_data[all_data['ID'].isin(df_result['ID'])].index).reset_index().drop(['index'], axis=1)
test_data = all_data[all_data['ID'].isin(df_result['ID'])].drop(['发电量'], axis=1).reset_index().drop(['index'], axis=1)
len(train_data), len(test_data)
# 去除重复值
train_data = train_data.drop_duplicates(train_data.columns.drop('ID'), keep='first')

In [8]:
train_data = add_avg(train_data)
test_data = add_avg(test_data)
cleaned_train_data = add_avg(cleaned_train_data)
cleaned_sub_data = add_avg(cleaned_sub_data)

In [9]:
X_train, X_test, y_train, y_test, sub_data, sm, polynm = generate_train_data(train_data, test_data, poly=True, select=True)

clean_X_train, clean_X_test, clean_y_train, clean_y_test, clean_sub_data, _, _ = generate_train_data(cleaned_train_data, cleaned_sub_data, poly=False, select=False)

clean_X = np.concatenate([clean_X_train, clean_X_test])
clean_y = np.concatenate([clean_y_train, clean_y_test])
clean_X = polynm.transform(clean_X)
clean_X = sm.transform(clean_X)

clean_sub_data = polynm.transform(clean_sub_data)
clean_sub_data = sm.transform(clean_sub_data)

In [12]:
all_X_train = np.concatenate([X_train, X_test])
all_y_train = np.concatenate([y_train, y_test])

In [18]:

lgb1 = LGBMRegressor(n_estimators=300, num_leaves=31, random_state=5,learning_rate=0.05) 
lgb2 = LGBMRegressor(n_estimators=150, num_leaves=31, random_state=5,learning_rate=0.05) 
lgb3 = LGBMRegressor(n_estimators=300, num_leaves=15, random_state=9,learning_rate=0.05)

lgb4 = LGBMRegressor(n_estimators=900, max_depth=5, random_state=5,learning_rate=0.05) 
lgb5 = LGBMRegressor(n_estimators=850, max_depth=4, random_state=7,learning_rate=0.05)
lgb6 = LGBMRegressor(n_estimators=720, max_depth=4, random_state=9,learning_rate=0.05)

lgb7 = LGBMRegressor(n_estimators=150, num_leaves=31, random_state=5,learning_rate=0.05,colsample_bytree=1,subsample=0.7) 
lgb8 = LGBMRegressor(n_estimators=300, num_leaves=31, random_state=7,learning_rate=0.05,colsample_bytree=0.7,subsample=0.7)
lgb9 = LGBMRegressor(n_estimators=300, num_leaves=15, random_state=9,learning_rate=0.05,colsample_bytree=0.7,subsample=1)

lgb10 = LGBMRegressor(n_estimators=900, num_leaves=31, random_state=5,learning_rate=0.05,colsample_bytree=1,subsample=0.7) 
lgb11 = LGBMRegressor(n_estimators=850, num_leaves=31, random_state=7,learning_rate=0.05,colsample_bytree=0.7,subsample=0.7)
lgb12 = LGBMRegressor(n_estimators=720, num_leaves=15, random_state=9,learning_rate=0.05,colsample_bytree=0.7,subsample=1)

gbdt1 = GradientBoostingRegressor(n_estimators=800, max_depth=4, max_features='log2', random_state=123,learning_rate=0.08)
gbdt2 = GradientBoostingRegressor(n_estimators=900, max_depth=4, max_features='log2', random_state=456,learning_rate=0.08)
gbdt3 = GradientBoostingRegressor(n_estimators=1000, max_depth=5, max_features='log2', random_state=789,learning_rate=0.08)

In [20]:
cross_validation_test(
    models=[lgb1,lgb2,lgb3,lgb4,lgb5,lgb6,
            lgb7,lgb8,lgb9,lgb10,lgb11,lgb12,
           gbdt1, gbdt2, gbdt3],
    train_X_data=[
        all_X_train, all_X_train, all_X_train, all_X_train,
        all_X_train, all_X_train, all_X_train, all_X_train,
        all_X_train, all_X_train, all_X_train,
        all_X_train, all_X_train, all_X_train, all_X_train],
    train_y_data=[
        all_y_train, all_y_train, all_y_train, all_y_train,
        all_y_train, all_y_train, all_y_train, all_y_train,
        all_y_train, all_y_train, all_y_train,
        all_y_train, all_y_train, all_y_train, all_y_train],
    cv=10
)

1 - Model: LGBMRegressor
MSE: [ 0.00856549  0.00959755  0.01580017  0.02191627  0.00989485  0.01248199
  0.16713013  0.00890663  0.01647759  0.01235421]
Score: [ 0.91528997  0.9107741   0.88833714  0.8710487   0.90952679  0.89950477
  0.70981614  0.91376363  0.88623801  0.8999689 ]
Average XGB - MSE: 0.0283124873952  - Score: 0.880426816945 

2 - Model: LGBMRegressor
MSE: [ 0.00899142  0.01009056  0.0168895   0.02207694  0.01082301  0.0129752
  0.16793271  0.00843897  0.01590355  0.01332501]
Score: [ 0.91338956  0.90871769  0.88498737  0.87063792  0.90576945  0.89773961
  0.70932251  0.91586509  0.88801328  0.89651201]
Average XGB - MSE: 0.028744688152  - Score: 0.879095451122 

3 - Model: LGBMRegressor
MSE: [ 0.00992164  0.01020999  0.01932998  0.02289779  0.010138    0.01219725
  0.16664599  0.00890462  0.01680996  0.01403937]
Score: [ 0.90941546  0.90822853  0.87793822  0.86856814  0.90852295  0.90054298
  0.71011482  0.91377251  0.8852274   0.89406424]
Average XGB - MSE: 0.02910946

KeyboardInterrupt: 

In [21]:
regrs = [
lgb1,lgb2,lgb3,lgb4,lgb5,lgb6,
            lgb7,lgb8,lgb9,lgb10,lgb11,lgb12,
           gbdt1, gbdt2, gbdt3
]

In [22]:
class Stacker(object):
    def __init__(self, n_splits, stacker, base_models):
        self.n_splits = n_splits
        self.stacker = stacker
        self.base_models = base_models
    
    # X: 原始训练集, y: 原始训练集真实值, predict_data: 原始待预测数据
    def fit_predict(self, X, y, predict_data):
        X = np.array(X)
        y = np.array(y)
        T = np.array(predict_data)

        folds = list(KFold(n_splits=self.n_splits, shuffle=False, random_state=2018).split(X, y))
        
        # 以基学习器预测结果为特征的 stacker的训练数据 与 stacker预测数据
        S_train = np.zeros((X.shape[0], len(self.base_models)))
        S_predict = np.zeros((T.shape[0], len(self.base_models)))
        
        for i, regr in enumerate(self.base_models):
            print(i + 1, 'Base model:', str(regr).split('(')[0])
            S_predict_i = np.zeros((T.shape[0], self.n_splits))
            
            for j, (train_idx, test_idx) in enumerate(folds):
                # 将X分为训练集与测试集
                X_train, y_train, X_test, y_test = X[train_idx], y[train_idx], X[test_idx], y[test_idx]
                print ('Fit fold', (j+1), '...')
                regr.fit(X_train, y_train)
                y_pred = regr.predict(X_test)                
                S_train[test_idx, i] = y_pred
                S_predict_i[:, j] = regr.predict(T)
            
            S_predict[:, i] = S_predict_i.mean(axis=1)

        nmse_score = cross_val_score(self.stacker, S_train, y, cv=5, scoring='neg_mean_squared_error')
        print('CV MSE:', -nmse_score)
        print('Stacker AVG MSE:', -nmse_score.mean(), 'Stacker AVG Score:', np.mean(np.divide(1, 1 + np.sqrt(-nmse_score))))

        self.stacker.fit(S_train, y)
        res = self.stacker.predict(S_predict)
        return res, S_train, S_predict

In [ ]:
# stacking_mode1 = Ridge(alpha=0.008, copy_X=True, fit_intercept=False, solver='auto', random_state=2)
stacking_model = SVR(C=100, gamma=0.01, epsilon=0.01)
stacker = Stacker(10, stacking_model, regrs)
pred_stack, S_train_data, S_predict_data = stacker.fit_predict(all_X_train, all_y_train, sub_data)

1 Base model: LGBMRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
Fit fold 6 ...
Fit fold 7 ...
Fit fold 8 ...
Fit fold 9 ...
Fit fold 10 ...
2 Base model: LGBMRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
Fit fold 6 ...
Fit fold 7 ...
Fit fold 8 ...
Fit fold 9 ...
Fit fold 10 ...
3 Base model: LGBMRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
Fit fold 6 ...
Fit fold 7 ...
Fit fold 8 ...
Fit fold 9 ...
Fit fold 10 ...
4 Base model: LGBMRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
Fit fold 6 ...
Fit fold 7 ...
Fit fold 8 ...
Fit fold 9 ...
Fit fold 10 ...
5 Base model: LGBMRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
Fit fold 6 ...
Fit fold 7 ...
Fit fold 8 ...
Fit fold 9 ...
Fit fold 10 ...
6 Base model: LGBMRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
Fi

In [ ]:
# stacking_mode1 = Ridge(alpha=0.008, copy_X=True, fit_intercept=False, solver='auto', random_state=2)
stacking_model = SVR(C=100, gamma=0.01, epsilon=0.01)
stacker = Stacker(10, stacking_model, regrs)
pred_stack, S_train_data, S_predict_data = stacker.fit_predict(all_X_train, all_y_train, sub_data)

1 Base model: LGBMRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
Fit fold 6 ...
Fit fold 7 ...
Fit fold 8 ...
Fit fold 9 ...
Fit fold 10 ...
2 Base model: LGBMRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
Fit fold 6 ...
Fit fold 7 ...
Fit fold 8 ...
Fit fold 9 ...
Fit fold 10 ...
3 Base model: LGBMRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
Fit fold 6 ...
Fit fold 7 ...
Fit fold 8 ...
Fit fold 9 ...
Fit fold 10 ...
4 Base model: LGBMRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
Fit fold 6 ...
Fit fold 7 ...
Fit fold 8 ...
Fit fold 9 ...
Fit fold 10 ...
5 Base model: LGBMRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
Fit fold 6 ...
Fit fold 7 ...
Fit fold 8 ...
Fit fold 9 ...
Fit fold 10 ...
6 Base model: LGBMRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
Fi

In [ ]:
df_result['score'] = pred_stack
index = df_result[df_result['ID'].isin(special_missing_ID)].index
df_result.loc[index, 'score'] = 0.379993053
c_index = df_result[df_result['ID'].isin(cleaned_sub_data_ID)].index
df_result.loc[c_index, 'score'] = pred_clean_stack
df_result.to_csv('sub_081902.csv', index=False, header=False)